In [2]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from collections import defaultdict
from gensim import corpora
from gensim.corpora import Dictionary

C:\Users\Yunmeng\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import nltk
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh import palettes
from bokeh.plotting import figure
from bokeh.models import HoverTool

In [4]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec
from gensim import models,utils

In [143]:
df = pd.read_csv("Indiegogo1.csv")
df["contents"] = df["title"]+df["tagline"]
df = df[["category","project_type","contents","funds_raised_percent"]].copy()

# group by percent

In [144]:
df1 = df[df["funds_raised_percent"]>=1]
df0 = df[df["funds_raised_percent"]<1]

# group by category

In [145]:
groups0 = df.groupby("category").groups
groups = dict()
for i in groups0.keys():
    groups[i] = df.iloc[groups0[i]]

In [146]:
category = df["category"].unique()

In [147]:
category

array(['Art', 'Comics', 'Dance & Theater', 'Film', 'Music', 'Photography',
       'Podcasts, Blogs & Vlogs', 'Tabletop Games', 'Audio',
       'Camera Gear', 'Energy & Green Tech', 'Fashion & Wearables',
       'Food & Beverages', 'Video Games', 'Web Series & TV Shows',
       'Writing & Publishing', 'Home', 'Phones & Accessories',
       'Productivity', 'Health & Fitness', 'Transportation',
       'Travel & Outdoors', 'Animal Rights', 'Culture', 'Education',
       'Environment', 'Human Rights', 'Local Businesses', 'Spirituality',
       'Wellness'], dtype=object)

In [148]:
arts=['Art', 'Comics', 'Dance & Theater', 'Film', 'Music', 'Photography', 'Fashion & Wearables']
leisure=['Podcasts, Blogs & Vlogs', 'Tabletop Games','Audio', 'Video Games', 'Web Series & TV Shows',
       'Camera Gear']
culture = ['Writing & Publishing','Animal Rights', 'Culture', 'Education',
       'Environment', 'Human Rights', 'Local Businesses', 'Spirituality',
       'Wellness']
tech = ['Energy & Green Tech','Camera Gear','Phones & Accessories','Productivity','Transportation',]
life = [ 'Food & Beverages','Home','Health & Fitness','Travel & Outdoors',]

In [149]:
for i in df["category"]:
    if 
d1 = df[df["category"] == arts[0]]
d1["label"] = 0

C:\Users\Yunmeng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# group by project type

In [10]:
campaign = df[df["project_type"]=="campaign"]
product = df[df["project_type"]=="product"]

# text preprocessing

In [11]:
stopwords = stopwords.words('english')
stopwords.append("")

In [12]:
def textPreprocess(content):
    contents0 = content
    length = len(contents0)
    contents = [re.sub('[^a-zA-Z]'," ",str(i)) for i in contents0]
    text0 = [sent.split(" ") for sent in contents]
    text1 = [[i.lower() for i in sent] for sent in text0]
    text2 = [[i for i in sent if i not in stopwords] for sent in text1]
    text3 = [" ".join(i) for i in text2]
    words = []
    for i in text2:
        words+=i
    fd = nltk.FreqDist(words).most_common()
    return fd,text2,text3

In [13]:
fd0 = textPreprocess(df["contents"])[0]

In [15]:
commons = [w for (w,c) in fd0[:40]]

In [16]:
output_notebook()

Loading BokehJS ...

# words Freq Plotting

In [151]:
def freqPlot(fd,title=""):
    wordsList = [w for (w,c) in fd]
    wordCounts = [c for (w,c) in fd]
    x_range = wordsList[20:50]
    p=figure(x_range =x_range,plot_height=270,plot_width=1200,
             background_fill_color="#e5d8bd",background_fill_alpha=0.2,
            title = title)
    p.add_tools(HoverTool())
    p.circle(x_range,wordCounts[20:50],size=8,fill_color = "#8da0cb",fill_alpha=1.0,
             line_color="#b3b3b3")
    p.line(x_range,wordCounts[20:50],line_width=2,line_color = "#66c2a5",line_alpha=0.9)
    return p

# percent plotting

In [152]:
from bokeh.layouts import column

In [206]:
fd0,words,sents = textPreprocess(df1["contents"])
fd = [(w,c) for (w,c) in fd0 if w not in commons]
p1 = freqPlot(fd,"Successful Projects words Frequency")

In [207]:
fd0,words,sents = textPreprocess(df0["contents"])
fd = [(w,c) for (w,c) in fd0 if w not in commons]
p2 = freqPlot(fd,"Failed Projects words frequency")

# category Plotting

In [192]:
fDist = dict()
catWords=[]
for i in range(30):
    fd0 = textPreprocess(groups[category[i]]["contents"])[0]
    fd = [(w,c) for (w,c) in fd0 if w not in commons]
    fDist[i] = fd[:15]
    catWords.append([w for (w,c) in fd[:15]])

In [193]:
Words = []
for i in catWords:
    Words+=i

In [194]:
dic = dict()

In [195]:
for i in range(30):
    dfc=pd.DataFrame([c for (w,c) in fDist[0]],index=[w for (w,c) in fDist[i]])
    dic[i] = dfc.reindex(Words).fillna(0).copy()

In [196]:
colors = palettes.Paired[12]+palettes.Set3[12]+palettes.Spectral[11]

In [197]:
s3 = palettes.Set3[12]

In [205]:
p4 = figure(x_range=(-10,600),y_range=(15,100),plot_height=500,plot_width=1500,
            background_fill_color=s3[6],background_fill_alpha=0.3)
for i in range(15):
    p4.circle(range(450),dic[i][0],color =colors[i],line_color="gray",line_width=1,
              legend=category[i],alpha=0.95,size=10,line_alpha=0.65)
#p2.legend.orientation = "horizontal,
p4.add_tools(HoverTool())
show(p4)

In [117]:
x_range=dic[0][0].index.tolist()

# by project

In [27]:
camp = textPreprocess(campaign["contents"])[0]
prod = textPreprocess(product["contents"])[0]

In [28]:
camp = [(w,c) for (w,c) in camp if w not in commons]
prod = [(w,c) for (w,c) in prod if w not in commons]

In [29]:
camp = camp[:300]
prod = prod[:300]

In [30]:
cw = [w for (w,c) in camp]
pw = [w for (w,c) in prod]

In [31]:
cp = cw+pw

In [32]:
c = pd.DataFrame([c for (w,c) in camp],index=[w for (w,c)in camp])
c = c.reindex(cp).fillna(0).copy()

In [33]:
p = pd.DataFrame([c for (w,c) in prod],index=[w for (w,c)in prod])
p = p.reindex(cp).fillna(0).copy()

In [212]:
p3 = figure(plot_height=300,plot_width=900,title = "Words Frequency by product types",
            background_fill_color=s3[8],background_fill_alpha=0.6)
p3.line(range(len(cp)),c[0],line_color =colors[0],legend="Campaign",line_width=1.5)
p3.line(range(len(cp)),p[0],line_color =colors[4],legend="Product",line_width=1.5)
p3.circle(range(len(cp)),c[0],color =colors[0])
p3.circle(range(len(cp)),p[0],color =colors[4])

GlyphRenderer(id='f2930ef0-e3f2-4a27-985d-441a3c05d25a', ...)

In [213]:
show(column(p1,p2,p3))